# Cómo construir un Chatbot avanzado con memoria de sesión usando LangChain
* Aplicación Chatbot LLM Avanzada.
    * Podrá tener una conversación.
    * Recordará interacciones previas: tendrá memoria.
    * Podrá tener diferentes memorias para diferentes sesiones de usuario.
    * Podrá recordar un número limitado de mensajes: memoria limitada.


## Conceptos incluidos
* Modelo de Chat vs. Modelo LLM:
    * El Modelo de Chat se basa en mensajes.
    * El Modelo LLM se basa en texto sin formato.
* Historial de Chat: permite que el Modelo de Chat recuerde interacciones previas.

## Configuración

#### Después de descargar el código del repositorio de github en tu computadora
En la terminal:
* cd nombre_del_proyecto
* pyenv local 3.11.4
* poetry install
* poetry shell

#### Para abrir el notebook con Jupyter Notebooks
En la terminal:
* jupyter lab

Ve a la carpeta de notebooks y abre el notebook correcto.

#### Para ver el código en Virtual Studio Code o tu editor de preferencia.
* abre Virtual Studio Code o tu editor de preferencia.
* abre la carpeta del proyecto
* abre el archivo 002-advanced-chatbot.py

## Crea tu archivo .env
* En el repositorio de github hemos incluido un archivo llamado .env.example
* Renombra ese archivo a .env y aquí es donde agregarás tus claves de API confidenciales. Recuerda incluir:
* OPENAI_API_KEY=tu_clave_de_api_de_openai
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=tu_clave_de_api_de_langchain
* LANGCHAIN_PROJECT=tu_nombre_de_proyecto

Llamaremos a nuestro proyecto LangSmith **002-advanced-chatbot**.

## Truco para evitar las molestas advertencias de depreciación de LangChain

En este ejercicio usaremos la cadena heredada LLMChain de LangChain. Funciona bien, pero LangChain muestra una molesta advertencia de depreciación. Para evitarlo, ingresaremos el siguiente código:

In [ ]:
import warnings
from langchain._api import LangChainDeprecationWarning

# Filtra las advertencias de depreciación de LangChain para evitar que se muestren
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Conéctate con el archivo .env ubicado en el mismo directorio de este notebook

Si estás usando el shell de poetry pre-cargado, no necesitas instalar el siguiente paquete porque ya está pre-cargado para ti:

In [ ]:
#!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# Carga las variables de entorno desde el archivo .env
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Instala LangChain

Si estás usando el shell de poetry pre-cargado, no necesitas instalar el siguiente paquete porque ya está pre-cargado para ti:

In [ ]:
#!pip install langchain

## Conéctate con un LLM y comienza una conversación con él

Si estás usando el shell de poetry pre-cargado, no necesitas instalar el siguiente paquete porque ya está pre-cargado para ti:

In [ ]:
#!pip install langchain-openai

* Para este proyecto, usaremos OpenAI's gpt-3.5-turbo

In [ ]:
from langchain_openai import ChatOpenAI

# Inicializa el chatbot con el modelo gpt-3.5-turbo de OpenAI
chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Podemos usar un modelo local con ollama

In [4]:
from langchain_ollama import ChatOllama

chatbot = ChatOllama(model='gpt-oss:20b')

* Mensaje Humano: la entrada del usuario.

In [ ]:
from langchain_core.messages import HumanMessage

# Define una lista de mensajes para enviar al chatbot
messagesToTheChatbot = [
    HumanMessage(content="Mi color favorito es el azul."),
]

#### Llama al ChatModel (el LLM)

In [ ]:
# Invoca al chatbot con los mensajes definidos
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and peaceful color. It's often associated with stability and trustworthiness. What do you like most about the color blue?", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-484b5288-6948-495f-88a1-cdd01e48f626-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43})

#### Rastrea la operación en LangSmith
* [Abre LangSmith aquí](https://smith.langchain.com)

## Verifica si el Chatbot recuerda tu color favorito.

In [ ]:
# Invoca al chatbot para preguntar cuál es el color favorito
chatbot.invoke([
    HumanMessage(content="¿Cuál es mi color favorito?")
])

AIMessage(content="I'm sorry, I'm not able to know your favorite color as an AI assistant. Can you please tell me what your favorite color is?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 13, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-75666902-eef4-4507-ba38-75b0b7e8a886-0', usage_metadata={'input_tokens': 13, 'output_tokens': 29, 'total_tokens': 42})

* Como puedes ver, nuestro Chatbot no puede recordar nuestra interacción previa.

## Agreguemos memoria a nuestro Chatbot
* Usaremos el paquete ChatMessageHistory.
* Guardaremos la memoria del Chatbot en un diccionario de python llamado chatbotMemory.
* Definiremos la función get_session_history para crear un session_id para cada conversación.
* Usaremos el runnable incorporado RunnableWithMesageHistory.

Si estás usando el shell de poetry pre-cargado, no necesitas instalar el siguiente paquete porque ya está pre-cargado para ti:

In [ ]:
#!pip install langchain_community

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Diccionario para almacenar la memoria del chatbot por sesión
chatbotMemory = {}

# Función para obtener el historial de chat de una sesión
# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # Si no existe una entrada para este session_id, crea una nueva
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]

# Crea un chatbot con historial de mensajes
chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

#### ¿Qué es BaseChatMessageHistory y qué hace?
BaseChatMessageHistory es lo que se llama una **clase base abstracta** en Python. [Ve más información sobre esto aquí](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). Esto significa que sirve como una plantilla o un **plano** fundacional para otras clases. Describe un conjunto de métodos y estructuras que cualquier clase que herede de ella debe implementar o cumplir, pero no se puede usar para crear objetos directamente.

Aquí hay un desglose más simple de lo que significa que `BaseChatMessageHistory` sea una clase base abstracta:

1. **Plano para otras clases:** Proporciona una estructura predefinida que otras clases pueden seguir. Piensa en ello como un esquema o una lista de verificación para construir algo; especifica lo que debe incluirse, pero no es el producto final.

2. **No se pueden crear instancias:** No puedes crear una instancia de una clase base abstracta. Intentar crear un objeto directamente desde `BaseChatMessageHistory` resultaría en un error porque está destinado a ser una guía, no algo para usar directamente.

3. **Requiere implementación:** Cualquier clase que herede de esta clase base abstracta necesita implementar métodos específicos descritos en `BaseChatMessageHistory`, como métodos para agregar mensajes, recuperar mensajes y borrar mensajes. La clase establece las reglas, y las subclases deben seguir estas reglas proporcionando los detalles operativos reales.

4. **Propósito en el diseño:** Usar una clase base abstracta ayuda a garantizar la coherencia y la corrección en la implementación de clases que la extienden. Es una forma de hacer cumplir ciertas funcionalidades en cualquier subclase, asegurándose de que todas se comporten como se espera sin reescribir el mismo código varias veces.

En general, el concepto de una clase base abstracta se trata de establecer estándares y reglas, dejando los detalles específicos de la ejecución a ser definidos por las subclases que heredan de ella.

#### Expliquemos el código anterior en términos simples
El código anterior gestiona la memoria del chatbot de las conversaciones basándose en identificadores de sesión. Aquí hay un desglose de lo que hacen los diferentes componentes:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: Esto inicializa un diccionario vacío donde se almacenarán los IDs de sesión y sus respectivos historiales de chat.

2. **Función get_session_history**:
    - Esta función, `get_session_history`, toma un `session_id` como argumento y devuelve el historial de chat asociado con esa sesión.
    - Si un historial de chat para el `session_id` dado no existe en `chatbotMemory`, se crea una nueva instancia de `ChatMessageHistory` y se asigna a ese `session_id` en el diccionario.
    - La función asegura que cada sesión tenga su propio historial de chat único, almacenado y recuperado usando el ID de sesión.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: Esta línea crea una instancia de `RunnableWithMessageHistory` usando dos argumentos: `chatbot` y `get_session_history`.
    - El `chatbot` se pasa junto con la función `get_session_history`. Esta configuración integra el chatbot con la funcionalidad para manejar historiales de chat específicos de la sesión, permitiendo que el chatbot mantenga la continuidad y el contexto en las conversaciones a través de diferentes sesiones.
    - **Aprende más sobre RunnableWithMessageHistory** [aquí](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

En general, el código organiza y gestiona la memoria de un chatbot, permitiéndole manejar diferentes sesiones con los usuarios de manera efectiva al recordar mensajes previos dentro de cada sesión.

#### RunnableWithMessageHistory
**Al invocar un nuevo RunnableWithMessageHistory, especificamos el historial de chat correspondiente usando un parámetro configurable**. Digamos que queremos crear una memoria de chat para una sesión de usuario, llamémosla sesión1:

In [ ]:
# Define la configuración para la sesión 1
session1 = {"configurable": {"session_id": "001"}}

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi color favorito es el rojo.")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

"That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?"

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'Your favorite color is red!'

## Cambiemos ahora el session_id y veamos qué pasa

Ahora creemos una memoria de chat para otra sesión de usuario, llamémosla sesión2:

In [ ]:
# Define la configuración para la sesión 2
session2 = {"configurable": {"session_id": "002"}}

Si el chatbot está usando esta nueva memoria para la sesión2, no podrá recordar nada de la conversación anterior en la sesión1:

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 2
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session2,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

"I'm sorry, I don't have that information. Can you please tell me what your favorite color is?"

## Volvamos a la sesión1 y veamos si la memoria todavía está ahí

In [ ]:
# Define la configuración para la sesión 1
session1 = {"configurable": {"session_id": "001"}}

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'Your favorite color is red!'

Como podemos ver, el chatbot ahora puede recordar la conversación de la sesión1.

## Nuestro ChatBot ahora tiene memoria de sesión. Comprobemos si recuerda la conversación de la sesión2.

In [ ]:
# Define la configuración para la sesión 2
session2 = {"configurable": {"session_id": "002"}}

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 2
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi nombre es Julio.")],
    config=session2,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

"Hello Julio! It's nice to meet you. If you'd like to share your favorite color with me, I'd be happy to remember it for future reference."

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 2
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi nombre?")],
    config=session2,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'Your name is Julio.'

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'Your favorite color is red!'

## Nuestro chatBot ahora recuerda cada una de nuestras conversaciones.

## La importancia de gestionar el historial de conversación
* La memoria de un chatbot se incluye en la ventana de contexto del LLM, por lo que, si no se gestiona, puede desbordarla.
* **Ahora vamos a aprender cómo limitar el tamaño de la memoria de un chatbot**.
* Primero, echemos un vistazo a lo que hay en la memoria de nuestro chatbot:

In [ ]:
# Imprime el contenido de la memoria del chatbot
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=['Human: My favorite color is red.', AIMessage(content="That's a bold and vibrant choice! Red is often associated with energy, passion, and strength. Do you have a specific reason why red is your favorite color?", response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5fde172-58ef-4f73-b463-a2a463d0a1f7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46}), AIMessage(content='Your favorite color is red!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 62, 'total_tokens': 68}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99eebba4-d387-4a22-b717-1180116d043a-0', usage_metadata={'input_tokens': 62, 'output_tokens': 6, 'total_tokens': 68}), AIMessage(content='Your favor

* Ahora, **definamos una función para limitar el número de mensajes almacenados en la memoria y agreguémosla a nuestra cadena con .assign**.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# Función para limitar el número de mensajes almacenados
def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

# Define un prompt para el chatbot
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente útil. Responde todas las preguntas lo mejor que puedas.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Crea una cadena con memoria limitada
limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

* La función limited_memory_of_messages te permite recortar la lista de mensajes almacenados, manteniendo solo un número específico de los últimos. Por ejemplo, si tienes una lista larga de mensajes y solo quieres mantener los últimos dos, esta función hará eso por ti.
* La función lambda funciona en conjunto con la función `limited_memory_of_messages`. Aquí hay un desglose simple:

    1. **Función Lambda**: La palabra clave `lambda` se usa para crear una pequeña función anónima en Python. La función `lambda` definida aquí toma un argumento, `x`.

    2. **Argumento de la función**: Se espera que el argumento `x` sea un diccionario que contenga una clave llamada `"messages"`. El valor asociado con esta clave es una lista de mensajes.

    3. **Cuerpo de la función**: El cuerpo de la función `lambda` llama a la función `limited_memory_of_messages`. Pasa la lista de mensajes encontrada en `x["messages"]` a esta función.

    4. **Comportamiento predeterminado de limited_memory_of_messages**: Dado que la función `lambda` no especifica el parámetro `number_of_messages_to_keep` cuando llama a `limited_memory_of_messages`, esta última tomará por defecto los últimos 2 mensajes de la lista (como se define en la función anterior).

En esencia, la función `lambda` es una forma abreviada de aplicar la función `limited_memory_of_messages` a la lista de mensajes contenida dentro de un diccionario. Recorta automáticamente la lista a los últimos dos mensajes.

**Creemos ahora nuestro nuevo chatbot con historial de mensajes limitado**:

In [ ]:
# Crea un chatbot con historial de mensajes limitado
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

## Agreguemos 2 mensajes más a la conversación de la sesión1:

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mis vehículos favoritos son los scooters Vespa.")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

"That's a fun choice! Vespa scooters are known for their classic and stylish design. They are also great for getting around town and exploring new places. What is it about Vespa scooters that you like the most?"

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi ciudad favorita es San Francisco.")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'San Francisco is a beautiful city with its iconic landmarks like the Golden Gate Bridge, Alcatraz Island, and cable cars. The diverse culture, amazing food scene, and stunning views of the bay make it a favorite for many people. What do you love most about San Francisco?'

## La memoria del chatbot ahora tiene 4 mensajes. Comprobemos el Chatbot con memoria limitada.
* Recuerda, este chatbot solo recuerda los últimos 2 mensajes, por lo que si le preguntamos sobre el primer mensaje no debería recordarlo.

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes limitado y la configuración de la sesión 1
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="¿cuál es mi color favorito?")],
    },
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

"I'm sorry, but as an AI assistant, I don't have access to personal information about you, such as your favorite color. If you'd like to share your favorite color with me, I'd be happy to discuss it further."

* El chatbot con memoria limitada se ha comportado como esperábamos.

## Finalmente, comparemos la respuesta anterior con la proporcionada por el Chatbot con memoria ilimitada

In [ ]:
from langchain_core.messages import HumanMessage

# Invoca al chatbot con historial de mensajes y la configuración de la sesión 1
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿cuál es mi color favorito?")],
    config=session1,
)

# Imprime el contenido de la respuesta del chatbot
responseFromChatbot.content

'Your favorite color is red!'

* Como puedes ver, este chatbot recuerda nuestro primer mensaje.

## Cómo ejecutar el código desde Visual Studio Code
* En Visual Studio Code, ve al archivo 004-invoke-stream-batch.py
* En la terminal, asegúrate de estar en el directorio del archivo y ejecuta:
    * python 002-advanced-chatbot.py